# Instagram Graph API call

## 1. Get Credential (w/ dotenv)

To protect credential info, you have to make `.env` file  
Here's some examples  

1. INSTAGRAM_ACCESS_TOKEN
2. INSTAGRAM_BUSINESS_ACCOUNT
3. FB_APP_SECRET_CODE
4. FB_CLIENT_ID

In [47]:
import requests
import json
import os
from pprint import pprint as pp
from dotenv import load_dotenv

# load .env
load_dotenv()
IG_ACCESS_TOKEN=os.environ.get("IG_ACCESS_TOKEN")
FB_APP_SECRET_CODE=os.environ.get('FB_APP_SECRET_CODE')
IG_BUSINESS_ID=os.environ.get('IG_BUSINESS_ID')
IG_USER_NAME=os.environ.get('IG_USER_NAME')

def get_creds():
    creds = dict() 
    creds['access_token'] = IG_ACCESS_TOKEN   # access token for use with all api calls
    creds['client_secret'] = FB_APP_SECRET_CODE  # client secret from facebook app
    creds['graph_domain'] = 'https://graph.facebook.com/' # base domain for api calls
    creds['graph_version'] = 'v20.0' # version of api we are hitting
    creds['instagram_account_id'] = IG_BUSINESS_ID # user business account id
    creds['ig_username'] = IG_USER_NAME
    # with domain and version
    creds['endpoint_base'] = creds['graph_domain'] + creds['graph_version'] + '/' # base endpoint
    
    return creds

## 2. Create Media Object Response & Publish Media

### (Optional) Create Carousel post

Steps required for making a carousel post are.  
1. Upload every images or video you want to make a carousel of one by one and store the container_id in a list.  
2. Make another container that contains the container_id list.  
3. Publish the container.  

In [48]:
def create_media_object(params):
    """Create media `Container` object
    
    API Endpoint:
    - For image
        https://graph.facebook.com/
        v20.0/{ig-user-id}/media?image_url={image-url}&caption={caption}&access_token={access_token}
    
    - For video
        https://graph.facebook.com/
        v20.0/{ig-user-id}/media?video_url={video-url}&caption={caption}&access_token={access_token}

    Returns:
        json object: {id : media_container_object_id}
    """

    url = params['endpoint_base'] + params['instagram_account_id'] + '/media'

    endpointParams = dict()
    if params['media_type'] == 'image':
        endpointParams['image_url'] = params['media_url']
    else:
        endpointParams['video_url'] = params['media_url']
        endpointParams['media_type'] = params['media_type']
    
    endpointParams['caption'] = params['caption']
    endpointParams['access_token'] = params['access_token']
    
    response = requests.post(url, data=endpointParams)

    return response.json()



def publish_media(media_object_id, params):
    """ Publish `media container` object
    
    API Endpoint:
        https://graph.facebook.com/
        v20.0/{ig-user-id}/media_publish?creation_id={creation-id}&access_token={access_token}

    Returns:
        json object: {id: media_id}
    """

    url = params['endpoint_base'] + params['instagram_account_id'] + '/media_publish'

    endpointParams = dict()
    endpointParams['creation_id'] = media_object_id
    endpointParams['access_token'] = params['access_token']

    response = requests.post(url, data=endpointParams)
    
    print("\n---- IMAGE MEDIA OBJECT PUBLISHED ---- \n")
    print("\tID:") # label
    print("\t" + response.json()['id']) # id of the object

    return response.json()

#### Example Post

In [49]:
params = get_creds()

In [43]:
# Create Image Media Post
params=get_creds()
params['media_type'] = 'image' # 2types in input('image', 'video')
params['media_url'] = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSCK8YiyxtZvfmGrgrxdwbLG1z7o9VxdI5Nkw&s' # sample image with high 
params['caption'] = 'This image was posted through the Instagram Graph API with a python script, test3 \n  #안정성 테스트 #해치웠나?'

image_media_object_response = create_media_object(params)
image_media_id = image_media_object_response['id']
published_media_id = publish_media(image_media_id, params)

## 3. Get User Media & Comments

In [44]:
def get_user_media(params):
    """ Get User Media id
    
    API Endpoint:
        https://graph.facebook.com/
        v20.0/{ig-user-id}?fields=media&access_token={access_token}

    Returns:
        json object:
    
        {'data' : [{id : {media_id_1},
                    id : {media_id_2},
                    id : {media_id_3},]}
    """
    
    url = params['endpoint_base'] + params['instagram_account_id'] + '/media'
    
    endpointParams = dict()
    endpointParams['access_token'] = params['access_token']
    endpointParams['fields'] = 'media'
    response = requests.get(url, endpointParams)
    return response.json()

def get_media_comments(media_id, params):
    """  Get media comments

    API Endpoint:
        https://graph.facebook.com/
        v20.0/{media_id}/comments?access_token={access_token}
    
    Returns:
        json object:

        {'data' : [{'timestamp': {created_time},
                    'text': {comment_text_1},
                    'id' : {comment_id_1}},

                    {'timestamp': {created_time},
                    'text': {comment_text_2},
                    'id': {comment_id_2}}]}
    """
    url = params['endpoint_base'] + media_id + '/comments?'
    endpointParams = dict()
    endpointParams['access_token'] = params['access_token']

    response = requests.get(url, endpointParams)
    return response.json()

### 

In [75]:
from pprint import pprint as pp

my_media = get_user_media(params)



# get all media comments
comments_data = []

for data in my_media['data']:
    temp_media_id = data['id']
    comments_data.append(get_media_comments(temp_media_id, params))

pp(comments_data)

[{'data': [{'id': '18040103149803453',
            'text': '나도 써보자',
            'timestamp': '2024-07-18T07:06:27+0000'},
           {'id': '18011557040594782',
            'text': '댓글을 작성해봅시다',
            'timestamp': '2024-07-17T19:17:41+0000'}]},
 {'data': []},
 {'data': []},
 {'data': []}]


In [76]:
comments_data

[{'data': [{'timestamp': '2024-07-18T07:06:27+0000',
    'text': '나도 써보자',
    'id': '18040103149803453'},
   {'timestamp': '2024-07-17T19:17:41+0000',
    'text': '댓글을 작성해봅시다',
    'id': '18011557040594782'}]},
 {'data': []},
 {'data': []},
 {'data': []}]

In [74]:
comments_id = []

for media_comments in comments_data:
    for comment in media_comments['data']:
        comments_id.append(comment['id'])
    
comments_id

['18040103149803453', '18011557040594782']

## 4. Post Reply for `comment`

### Reply Sequence
- 1. Get replies in comment
- 2. Check - Is it already replied?
  - If yes) Is the last response was created by media creator? 
- 3. Get all texts in replies (Optional - TBD)
- 4. Create reply response (w/ LLM)
- 5. Post reply to a comment

In [46]:
def get_comment_replies(comment_id, params):
    """Create reply for comment
    
    API Endpoint:
        https://graph.facebook.com/
        v20.0/{ig_comment_id}/replies?fields={fields}&access_token={access-token}
    
    Returns:
        json object: the replies are `stacked-shaped`(LIFO)
        
        {'data': [{'id': {replied_id_2},
                'text': {replied_text_2}
                'username': {ig_username},

                {'id', {replied_id_1},
                'text': {replied_text_1},
                'username': {ig_username}
                ]}
    """

    url = params['endpoint_base'] + comment_id + '/replies'
    endpointParams = dict()
    endpointParams['access_token'] = params['access_token']
    endpointParams['fields'] = 'id, text, username'
    response = requests.get(url, endpointParams)
    return response.json()

In [51]:
def is_replied(replies, params):
    """Check if last response's username is creator's name

    Returns:
        True or False
    """
    if len(replies['data']):
        return True if replies['data'][0]['username'] == params['ig_username'] else False

In [58]:
print(temp_replies)
print(is_replied(temp_replies, params))

{'data': [{'id': '17997719987643254', 'text': '태그 없이도 답글이 가능할까?', 'username': 'analyst_hyuk'}, {'id': '17970962732744635', 'text': 'This reply was created through API call', 'username': 'analyst_hyuk'}]}
True


In [60]:
def get_reply_texts(replies, params):
    """
        일단 그냥 LLM에 넣어보고 잘 돌아가면 pass,
        - 필요할 경우 이후 작성
    """
    pass

In [33]:
def post_reply_to_comment(comment_id, params):
    """ Post reply to a comment

    API Endpoint:
        https://graph.facebook.com/
        v20.0/{ig_comment_id}/replies&access_token={access-token}

    Returns:
        Status Code <Response>
        - 200 : OK
        - 400 : Not supported input form
    """
    
    url = params['endpoint_base'] + comment_id + '/replies'

    endpointParams = dict()
    endpointParams['message'] = params['reply_message']
    endpointParams['access_token'] = params['access_token']
    response = requests.post(url, data=endpointParams)
    return response

In [77]:
params['reply_message'] = "This reply was created through API call"

# post_response = post_reply_to_comment(comments_id[0], params)

In [61]:
# 참고문서
"""
https://developers.facebook.com/docs/instagram-platform/instagram-graph-api/content-publishing#----------            # publishing

https://developers.facebook.com/docs/instagram-platform/instagram-graph-api/reference/ig-user/media_publish         # replied comment

https://velog.io/@shi9476/Instagram-API-%EC%97%B0%EB%8F%99               # 참고자료 1 - long live token 자동재발급

https://medium.com/@maanideeprkummiitha/a-beginners-guide-to-interacting-with-instagram-using-python-and-the-graph-api-8fe7956fcc   # create api function tutorial

https://developers.facebook.com/docs/instagram-platform/instagram-graph-api/comment-moderation""" # 댓글에 답글 작성


'\nhttps://medium.com/@maanideeprkummiitha/a-beginners-guide-to-interacting-with-instagram-using-python-and-the-graph-api-8fe7956fcc   # create api function tutorial\n\nhttps://velog.io/@shi9476/Instagram-API-%EC%97%B0%EB%8F%99               # 참고자료 1 - long live token 자동재발급\n\nhttps://developers.facebook.com/docs/instagram-platform/instagram-graph-api/content-publishing#----------            # publishing\n\nhttps://developers.facebook.com/docs/instagram-platform/instagram-graph-api/reference/ig-user/media_publish         # replied comment\n\nhttps://developers.facebook.com/docs/instagram-platform/instagram-graph-api/comment-moderation'

In [28]:
post_response

<Response [200]>